In [135]:
import requests, csv, os,time,re
from  bs4 import BeautifulSoup
url = "https://www.heffnermanagement.com/ashx/autocomplete.ashx"
basedir = 'heffnermanagement.com'
base_url = 'https://www.heffnermanagement.com'
models_details=list()

In [53]:
response = requests.get(url)

In [61]:
result = response.json()

In [74]:

models_dict={i["label"]:base_url+i['url'] for i in result if i['subname']=='Plus,'}

In [138]:
def download_pics(pics):
    for pic in pics:
        regex= re.search('\d+.jpg',pic).group(0)
        path = f"{basedir}/{k}/{regex}"
        if not os.path.exists(path):
            with open(path, 'wb') as f:
                f.write(requests.get(pic).content)


def get_model(model_id):
    milisec = int(time.time() * 1000)
    headers = {
        'authority': 'www.heffnermanagement.com',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'cookie': 'ASP.NET_SessionId=cvkvmglhppyeiel3qi4q5zkp',
        'dnt': '1',
        'referer': 'https://www.heffnermanagement.com/Portfolio/women/plus/women/1491251/adrianna-moss',
        'sec-ch-ua': '^\\^Microsoft',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '^\\^Windows^\\^',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'sec-gpc': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.42',
        'x-requested-with': 'XMLHttpRequest',
    }

    params = (
        ('getResultsFrom', '1'),
        ('getResultsTo', '24'),
        ('division', 'women'),
        ('board', 'plus'),
        ('gender', 'women'),
        ('modelid', model_id),
        ('page', 'portfolio'),
        ('indx', '0'),
        ('_', milisec),
    )

    response = requests.get('https://www.heffnermanagement.com/Ajax/getPortfolioImgs.aspx', headers=headers, params=params)
    return response.content

def list_to_csv(lst):
    with open(f"{basedir}/models_info.csv",'w') as f:
        c = csv.DictWriter(f, ["NAME","URL", "HEIGHT", "BUST","WAIST","HIP","DRESS" ,"EYE","HAIR", "SHOE"])
        c.writeheader()
        for row in lst:
            c.writerow(row)

In [ ]:
for k,v in models_dict.items():
    os.makedirs(f"{basedir}/{k}", exist_ok=True)
    print(k, v)
    model_id=re.findall(r'\d+',v)[0]
    soup = BeautifulSoup(requests.get(v).content, 'html5lib')
    feat_dict = {'NAME':k, "URL":v}
    feat_dict.update({i.text.split(':')[0].upper():i.text.split(':')[1] for i in soup.find('div', class_='modelStats').findAll('span')})
    soup = BeautifulSoup(get_model(model_id), 'html5lib')
    pics = [pic.img['src'] for pic in soup.findAll('div',class_='grid-item P')]
    download_pics(pics)
    models_details.append(feat_dict)




In [139]:
list_to_csv(models_details)